Reinforcement Learning for Large Language Models
Winter 23/24 Semester
Final Group Project
Kateryna Smykovska, Jakob Schmitter, Suvi Lehtosalo, Megan Horikawa

This notebook was made by **Megan Horikawa **

[QA - 7b Chat Notebook](https://colab.research.google.com/drive/1BB6PoeNeyqn01ezm_TZJoHwxAP-ig6-o?usp=sharing)

In [ ]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install torch
!pip install datasets
!pip install evaluate
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer
import torch


import datasets
import evaluate
import numpy as np

from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", token=my_secret_key)

model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_4bit=True,
    device_map="auto",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    token = my_secret_key
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Load Dataset for NLI task. Superglue subdataset rte was chosen
copa_dataset = datasets.load_dataset("super_glue", "copa")

#train subset was selected as it had over 150 entries with a mixture of both labels

dataset = copa_dataset['train']


In [ ]:
# define log likelihood function (taken from homework with small changes for llama)

def get_log_prob_of_completion(
        model,
        tokenizer,
        prompt,
        completion,
        device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
):
        """
        Convenience function for computing the log probability of a completion
        given a prompt.
        """
        # tokenize the prompt and the completion
        # truncate so as to fit into to maximal context window of llama2
        # which is 2048 tokens
        input_ids = tokenizer(
                prompt + completion,
                return_tensors='pt',
                truncation=True,
                max_length=2048,
        )['input_ids'].to(device)

        # separately tokenize prompt
        # so as to access the logits for the completion only
        # when scoring the completion
        input_ids_prompt = tokenizer(
                prompt,
                return_tensors='pt',
                truncation=True,
                max_length=2048
        )['input_ids'].to(device)

        # create attention mask and position ids
        attention_mask = (input_ids != tokenizer.eos_token_id).to(dtype=torch.int64)
        position_ids = attention_mask.cumsum(-1)-1
        # get the logits for the completion
        with torch.no_grad():
                out = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        position_ids=position_ids
                )

        # get the logits of the completion
        # for that, make a tensor of the logits
        # for the completion only
        # in particular, we shift the indices by one to the left to access logits of the
        # actual sequence tokens
        logits_completion = out.logits[:, :-1]
        logits_completion = logits_completion.squeeze()
        # get the log probabilities for the completion
        log_probs = torch.nn.functional.log_softmax(
                logits_completion,
                dim=-1
        )
        # retrieve the logit corresponding to the actual completion tokens
        try:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].squeeze().unsqueeze(-1)
                )
        except:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].unsqueeze(-1)
                )

        continuationConditionalLogProbs = log_completion_tokens[
                (input_ids_prompt.shape[-1]-1):
        ]
        completion_log_prob = torch.mean(
                continuationConditionalLogProbs
        ).cpu()

        return completion_log_prob

In [ ]:


#iterate over the dataset:

results = []
prompt_list = []
correct_option = []
incorrect_option = []
log_ps_correct_option = []
log_ps_incorrect_option = []

indx = 1

for item in dataset:
  if indx==151:
    break
  # concatenate the premise with each of the choices
  prompt = item['premise']
  # get classification of item as either entailment (0) or non-entailment(1)
  category = item['label']

  correct = ''
  incorrect = ''

  # set correct classification based on label
  if category == 0:
    correct = item['choice1']
    incorrect = item['choice2']

  else:
    correct = item['choice2']
    incorrect = item['choice1']

  prompt_list.append(prompt)
  correct_option.append(correct)
  incorrect_option.append(incorrect)

  #compute the lob probabilities for both entailment and non-entailment classes

  log_p_correct = get_log_prob_of_completion(
      model,
      tokenizer,
      prompt,
      correct
  )
  # append to the correct list
  log_ps_correct_option.append(log_p_correct)

  log_p_incorrect = get_log_prob_of_completion(
      model,
      tokenizer,
      prompt,
      incorrect
  )

  # append to the incorrect list
  log_ps_incorrect_option.append(log_p_incorrect)

  indx +=1


In [ ]:
# lets load things into pandas
import pandas as pd

df = pd.DataFrame(list(zip(prompt_list,correct_option, log_ps_correct_option,incorrect_option, log_ps_incorrect_option)), columns = ['prompt','correct', 'log_prob_correct','incorrect', 'log_prob_incorrect'])


# evaluate the log probabilities
# check whether the log probaility of the correct answer is higher than the
# incorrect answer and append to the results list
df['correct_prediction'] = df['log_prob_correct']> df['log_prob_incorrect']


df.head(20)


,prompt,correct,log_prob_correct,incorrect,log_prob_incorrect,correct_prediction
0,My body cast a shadow over the grass.,The sun was rising.,tensor(-4.4031),The grass was cut.,tensor(-4.9314),True
1,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,tensor(-2.3509),The woman felt that her friend took advantage ...,tensor(-2.4729),True
2,The women met for coffee.,They wanted to catch up with each other.,tensor(-2.6818),The cafe reopened in a new location.,tensor(-3.1916),True
3,The runner wore shorts.,The forecast predicted high temperatures.,tensor(-4.0954),She planned to run along the beach.,tensor(-3.7655),False
4,The guests of the party hid behind the couch.,It was a surprise party.,tensor(-3.3635),It was a birthday party.,tensor(-2.9453),False
5,The politician lost the election.,No one voted for him.,tensor(-2.7956),He ran negative campaign ads.,tensor(-4.6217),True
6,The stain came out of the shirt.,I bleached the shirt.,tensor(-3.7531),I patched the shirt.,tensor(-3.8162),True
7,The man got a discount on his groceries.,He used a coupon.,tensor(-3.4223),He greeted the cashier.,tensor(-2.8528),False
8,The physician misdiagnosed the patient.,The patient filed a malpractice lawsuit agains...,tensor(-1.4778),The patient disclosed confidential information...,tensor(-2.6107),True
9,The customer filed a complaint with the store ...,The sales associate acted rude to the customer.,tensor(-3.5550),The sales associate undercharged the customer.,tensor(-4.0301),True


In [ ]:
# export to csv
from google.colab import drive
drive.mount('/content/drive')

df.to_csv('/content/drive/My Drive/RLProject/QA_llama2_7b.csv', index=False)

Mounted at /content/drive


Link to csv file: https://drive.google.com/file/d/1hjInJMNmndr3XBWZGXyx4XHHVxz_5Uj_/view?usp=drive_link